In [26]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

# Set random seed
np.random.seed(0)

In [27]:
data = pd.read_csv('BS_data.csv')
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,Net Debt,Price
0,RHI,2013-12-01,1490.3,570.6,919.6,135.3,6.80,718.3,5.31,1.4,-274.3,41.99
1,RCL,2011-12-01,19804.4,11396.6,8407.8,217.5,38.74,7442.4,34.29,8507.2,8245.0,24.77
2,NVDA,2020-01-01,17315.0,5111.0,12204.0,612.0,19.93,11537.0,18.84,2643.0,-8254.0,236.43
3,STC,2012-12-01,1291.2,710.8,580.4,21.1,29.31,263.5,13.58,71.2,-137.4,26.00
4,XRAY,2010-12-01,3258.0,1348.0,1909.9,142.1,12.98,457.6,3.23,786.6,246.6,34.17
5,GWW,2016-12-01,5694.3,3788.5,1905.8,58.8,30.57,684.7,11.64,2247.1,1972.9,232.25
6,SKM,2015-12-01,24288.4,11223.5,13064.9,70.6,183.55,9386.2,132.93,7039.9,5790.9,20.15
7,BAX,2011-12-01,19073.0,12245.0,6828.0,560.3,11.74,3442.0,6.14,5195.0,2290.0,26.88
8,MAR,2012-12-01,6342.0,7627.0,-1285.0,312.3,-4.13,-3274.0,-10.53,2935.0,2847.0,37.27
9,SJM,2019-04-01,16711.3,8740.8,7970.5,113.7,70.08,-5059.2,-44.48,5959.9,5858.6,122.63


In [28]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)



Total Assets
Done
Total Liabilities
Done
Total Equity
Done
Total Shares Out. on Filing Date
Done
Book Value / Share
Done
Tangible Book Value
Done
Tangible Book Value Per Share
Done
Total Debt
Done
Net Debt
Done


In [29]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:9] # Parameters
y = NpMatrix[:,9] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))
number_of_features = len(X[1])

X: [[1.63843386e-03 6.82611560e-04 2.81693214e-02 ... 1.34060792e-03
  2.92521072e-06 1.85568632e-01]
 [2.19349988e-02 1.36816489e-02 4.51864494e-02 ... 1.49174313e-03
  1.77752519e-02 2.00645972e-01]
 [1.91761262e-02 6.13437854e-03 5.38134120e-02 ... 1.41116898e-03
  5.52237995e-03 1.71446269e-01]
 ...
 [6.79067888e-03 4.97604015e-03 3.06086506e-02 ... 1.34050362e-03
  4.01380699e-03 1.89340224e-01]
 [2.35835177e-03 1.36510305e-03 2.83540777e-02 ... 1.34947368e-03
  1.52340795e-03 1.86742708e-01]
 [7.21358632e-04 8.08087212e-04 2.60515545e-02 ... 1.31145520e-03
  1.21500717e-03 1.86722887e-01]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 9
Total Number of Training instances: 5000


[ 41.99  24.77 236.43 ...  49.01   8.07  34.69]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 41.99
Total number of y values 5000


In [30]:
#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=9, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))
    
    network.add(layers.Dense(units=9, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.2))

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [31]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size= 5)

In [32]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 3s 655us/sample - loss: 58.8265 - mae: 58.8265 - mse: 20120.2070
Epoch 2/1000
4000/4000 [==============================] - 2s 515us/sample - loss: 48.3519 - mae: 48.3519 - mse: 17994.1582
Epoch 3/1000
4000/4000 [==============================] - 2s 482us/sample - loss: 48.0644 - mae: 48.0644 - mse: 17933.2656
Epoch 4/1000
4000/4000 [==============================] - 2s 466us/sample - loss: 48.2032 - mae: 48.2032 - mse: 17825.5488
Epoch 5/1000
4000/4000 [==============================] - 2s 469us/sample - loss: 47.6547 - mae: 47.6547 - mse: 17938.1641
Epoch 6/1000
4000/4000 [==============================] - 2s 473us/sample - loss: 47.6980 - mae: 47.6979 - mse: 17831.0527
Epoch 7/1000
4000/4000 [==============================] - 2s 433us/sample - loss: 47.8008 - mae: 47.8008 - mse: 17958.0586
Epoch 8/1000
4000/4000 [==============================] - 2s 417us/sample - loss: 47.7615 - mae: 47.7615 - mse: 17976

4000/4000 [==============================] - 2s 387us/sample - loss: 44.5949 - mae: 44.5949 - mse: 16881.6621
Epoch 132/1000
4000/4000 [==============================] - 2s 385us/sample - loss: 44.6412 - mae: 44.6412 - mse: 16915.0293
Epoch 133/1000
4000/4000 [==============================] - 1s 345us/sample - loss: 44.4339 - mae: 44.4340 - mse: 16822.7676
Epoch 134/1000
4000/4000 [==============================] - 2s 434us/sample - loss: 44.7557 - mae: 44.7557 - mse: 17028.4043
Epoch 135/1000
4000/4000 [==============================] - 2s 379us/sample - loss: 44.5415 - mae: 44.5415 - mse: 16803.4395
Epoch 136/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 44.3780 - mae: 44.3781 - mse: 16915.5488
Epoch 137/1000
4000/4000 [==============================] - 2s 390us/sample - loss: 44.3288 - mae: 44.3289 - mse: 16811.6016
Epoch 138/1000
4000/4000 [==============================] - 2s 386us/sample - loss: 44.4036 - mae: 44.4036 - mse: 16725.6621
Epoch 139/1000


4000/4000 [==============================] - 2s 417us/sample - loss: 42.4806 - mae: 42.4806 - mse: 15784.8271
Epoch 262/1000
4000/4000 [==============================] - 2s 426us/sample - loss: 42.4735 - mae: 42.4735 - mse: 15733.6416
Epoch 263/1000
4000/4000 [==============================] - 2s 384us/sample - loss: 42.6153 - mae: 42.6153 - mse: 15768.3486
Epoch 264/1000
4000/4000 [==============================] - 2s 412us/sample - loss: 42.8196 - mae: 42.8196 - mse: 15914.1660
Epoch 265/1000
4000/4000 [==============================] - 2s 393us/sample - loss: 42.8272 - mae: 42.8272 - mse: 15934.4434
Epoch 266/1000
4000/4000 [==============================] - 2s 414us/sample - loss: 42.5936 - mae: 42.5936 - mse: 15777.7842
Epoch 267/1000
4000/4000 [==============================] - 2s 418us/sample - loss: 42.9403 - mae: 42.9403 - mse: 16067.5645
Epoch 268/1000
4000/4000 [==============================] - 2s 427us/sample - loss: 42.3163 - mae: 42.3163 - mse: 15879.6572
Epoch 269/1000


4000/4000 [==============================] - 2s 393us/sample - loss: 42.3309 - mae: 42.3310 - mse: 15621.5918
Epoch 391/1000
4000/4000 [==============================] - 2s 389us/sample - loss: 42.0005 - mae: 42.0005 - mse: 15629.5098
Epoch 392/1000
4000/4000 [==============================] - 2s 390us/sample - loss: 42.0741 - mae: 42.0740 - mse: 15459.6436
Epoch 393/1000
4000/4000 [==============================] - 2s 449us/sample - loss: 41.8818 - mae: 41.8818 - mse: 15482.8027s - los
Epoch 394/1000
4000/4000 [==============================] - 2s 378us/sample - loss: 42.5472 - mae: 42.5472 - mse: 15397.4502
Epoch 395/1000
4000/4000 [==============================] - 2s 387us/sample - loss: 42.0529 - mae: 42.0529 - mse: 15603.7832
Epoch 396/1000
4000/4000 [==============================] - 2s 402us/sample - loss: 42.1950 - mae: 42.1950 - mse: 15593.4775
Epoch 397/1000
4000/4000 [==============================] - 2s 390us/sample - loss: 41.9470 - mae: 41.9470 - mse: 15481.2656
Epoch 39

4000/4000 [==============================] - 2s 436us/sample - loss: 42.2118 - mae: 42.2118 - mse: 15558.4297
Epoch 521/1000
4000/4000 [==============================] - 2s 391us/sample - loss: 41.7897 - mae: 41.7896 - mse: 15467.2383
Epoch 522/1000
4000/4000 [==============================] - 2s 377us/sample - loss: 41.6515 - mae: 41.6515 - mse: 15579.8086
Epoch 523/1000
4000/4000 [==============================] - 2s 398us/sample - loss: 42.1786 - mae: 42.1787 - mse: 15560.6553
Epoch 524/1000
4000/4000 [==============================] - 2s 428us/sample - loss: 42.1134 - mae: 42.1134 - mse: 15811.6934
Epoch 525/1000
4000/4000 [==============================] - 2s 391us/sample - loss: 41.8381 - mae: 41.8381 - mse: 15609.6299
Epoch 526/1000
4000/4000 [==============================] - 2s 389us/sample - loss: 41.8219 - mae: 41.8219 - mse: 15299.5498
Epoch 527/1000
4000/4000 [==============================] - 2s 389us/sample - loss: 42.1749 - mae: 42.1749 - mse: 15531.2031
Epoch 528/1000


...

4000/4000 [==============================] - 1s 245us/sample - loss: 42.1562 - mae: 42.1562 - mse: 14919.1113- loss: 37.2772 - mae: 37.27
Epoch 585/1000
4000/4000 [==============================] - 1s 241us/sample - loss: 42.1623 - mae: 42.1624 - mse: 15078.5010
Epoch 586/1000
4000/4000 [==============================] - 1s 272us/sample - loss: 42.0190 - mae: 42.0190 - mse: 15144.4902
Epoch 587/1000
4000/4000 [==============================] - 1s 254us/sample - loss: 41.9222 - mae: 41.9222 - mse: 14934.8271
Epoch 588/1000
4000/4000 [==============================] - 1s 248us/sample - loss: 42.0858 - mae: 42.0858 - mse: 14818.8311s - loss: 47.6421 - mae: 47.6421 -
Epoch 589/1000
4000/4000 [==============================] - 1s 247us/sample - loss: 41.6817 - mae: 41.6817 - mse: 14912.3760
Epoch 590/1000
4000/4000 [==============================] - 1s 240us/sample - loss: 42.0710 - mae: 42.0710 - mse: 14935.7803
Epoch 591/1000
4000/4000 [==============================] - 1s 258us/sample - 

Epoch 648/1000
4000/4000 [==============================] - 1s 284us/sample - loss: 42.1281 - mae: 42.1281 - mse: 15269.5518
Epoch 649/1000
4000/4000 [==============================] - 1s 249us/sample - loss: 42.3425 - mae: 42.3425 - mse: 15290.6016s - loss: 43.5302 - mae: 43.5302 - mse: 17015.9
Epoch 650/1000
4000/4000 [==============================] - 1s 241us/sample - loss: 42.0010 - mae: 42.0010 - mse: 14996.1357
Epoch 651/1000
4000/4000 [==============================] - 1s 242us/sample - loss: 41.7237 - mae: 41.7237 - mse: 14909.3330
Epoch 652/1000
4000/4000 [==============================] - 1s 244us/sample - loss: 42.2232 - mae: 42.2232 - mse: 15088.0264
Epoch 653/1000
4000/4000 [==============================] - 1s 270us/sample - loss: 42.0734 - mae: 42.0734 - mse: 15206.2451
Epoch 654/1000
4000/4000 [==============================] - 1s 264us/sample - loss: 41.8170 - mae: 41.8170 - mse: 15206.2783
Epoch 655/1000
4000/4000 [==============================] - 1s 246us/sample - 

Epoch 711/1000
4000/4000 [==============================] - 1s 244us/sample - loss: 41.9477 - mae: 41.9477 - mse: 14718.2881
Epoch 712/1000
4000/4000 [==============================] - 1s 242us/sample - loss: 41.8980 - mae: 41.8980 - mse: 14728.2764s - loss: 41.1978 - mae: 41.1978 - mse: 
Epoch 713/1000
4000/4000 [==============================] - 1s 240us/sample - loss: 42.0739 - mae: 42.0739 - mse: 14878.4932
Epoch 714/1000
4000/4000 [==============================] - 1s 254us/sample - loss: 41.7819 - mae: 41.7819 - mse: 14764.6436
Epoch 715/1000
4000/4000 [==============================] - 1s 278us/sample - loss: 41.6829 - mae: 41.6829 - mse: 14801.9111
Epoch 716/1000
4000/4000 [==============================] - 1s 243us/sample - loss: 41.5533 - mae: 41.5533 - mse: 14855.3408- loss: 38.5403 - ma
Epoch 717/1000
4000/4000 [==============================] - 1s 243us/sample - loss: 41.7314 - mae: 41.7314 - mse: 14966.6670
Epoch 718/1000
4000/4000 [==============================] - 1s 24

4000/4000 [==============================] - 1s 275us/sample - loss: 41.5705 - mae: 41.5705 - mse: 14899.5352
Epoch 840/1000
4000/4000 [==============================] - 1s 244us/sample - loss: 42.3623 - mae: 42.3623 - mse: 15037.8037
Epoch 841/1000
4000/4000 [==============================] - 1s 240us/sample - loss: 42.1699 - mae: 42.1699 - mse: 14965.2920
Epoch 842/1000
4000/4000 [==============================] - 1s 246us/sample - loss: 41.7557 - mae: 41.7557 - mse: 14789.1562s - loss: 40.7999 - mae: 40.7999 - mse: 
Epoch 843/1000
4000/4000 [==============================] - 1s 254us/sample - loss: 41.9197 - mae: 41.9197 - mse: 14743.1182s - loss: 42.5034 - mae: 42.5034 - mse: 1
Epoch 844/1000
4000/4000 [==============================] - 1s 273us/sample - loss: 41.6559 - mae: 41.6559 - mse: 14833.2725
Epoch 845/1000
4000/4000 [==============================] - 1s 249us/sample - loss: 41.4959 - mae: 41.4959 - mse: 14818.6582
Epoch 846/1000
4000/4000 [==============================] -

4000/4000 [==============================] - 1s 252us/sample - loss: 41.4310 - mae: 41.4310 - mse: 14769.9316- loss: 40.1550 - mae: 40.1550 - mse: 
Epoch 965/1000
4000/4000 [==============================] - 1s 240us/sample - loss: 41.7720 - mae: 41.7719 - mse: 15057.3047
Epoch 966/1000
4000/4000 [==============================] - 1s 237us/sample - loss: 41.8094 - mae: 41.8094 - mse: 14993.4326
Epoch 967/1000
4000/4000 [==============================] - 1s 246us/sample - loss: 41.6872 - mae: 41.6872 - mse: 15046.2002
Epoch 968/1000
4000/4000 [==============================] - 1s 276us/sample - loss: 41.3727 - mae: 41.3727 - mse: 14788.5098
Epoch 969/1000
4000/4000 [==============================] - 1s 260us/sample - loss: 41.9466 - mae: 41.9466 - mse: 14692.3750
Epoch 970/1000
4000/4000 [==============================] - 1s 243us/sample - loss: 42.2563 - mae: 42.2562 - mse: 14908.5547
Epoch 971/1000
4000/4000 [==============================] - 1s 241us/sample - loss: 41.8589 - mae: 41.

In [33]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-47.77951225 -41.63351285 -40.83084797 -36.22175232 -38.09409297]
Mean MAE: -40.91194367265702
Standard Deviation of the mean: 3.9402770025745997
